In [1]:
from op_analytics.coreutils.duckdb_inmem.client import init_client
from op_analytics.coreutils.duckdb_inmem.localcopy import dump_local_copy, load_local_copy
from op_analytics.cli.subcommands.pulls.defillama.dataaccess import DefiLlama

duckdb_ctx = init_client()

2025-01-03 16:22:13 [info     ] initialized duckdb at /var/folders/tz/g2vwn0qx5ll32j7jfsshhps40000gn/T/gyjbsuq2/op-analytics.duck.db filename=client.py lineno=61 process=57051
2025-01-03 16:22:13 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=57051
2025-01-03 16:22:13 [debug    ] loaded vault: 18 items         filename=vault.py lineno=76 process=57051


In [2]:
## EXAMPLE 1: Register remote parquet files as a view.

# The reader returns the view name so it can be used in queries.
view1 = DefiLlama.PROTOCOLS_TOKEN_TVL.read(min_date="2024-10-01")
print(view1)

print(duckdb_ctx.client.sql(f"DESCRIBE {view1}"))



2025-01-03 16:22:14 [info     ] Reading data from 'defillama/protocols_token_tvl_v1' with filters min_date=2024-10-01, max_date=None, date_range_spec=None filename=dailydata.py lineno=176 process=57051
2025-01-03 16:22:14 [info     ] querying markers for 'defillama/protocols_token_tvl_v1' DateFilter(min_date=datetime.date(2024, 10, 1), max_date=None, datevals=None) filename=dailydata.py lineno=103 process=57051
2025-01-03 16:22:14 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=36 process=57051
2025-01-03 16:22:15 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=41 process=57051
2025-01-03 16:22:15 [info     ] 311 markers found              filename=dailydata.py lineno=117 max_dt=2025-01-03 min_dt=2024-10-01 process=57051
2025-01-03 16:22:15 [info     ] 95 distinct paths              filename=dailydata.py lineno=127 process=57051
2025-01-03 16:22:18 [info     ] registered view: 'defillama_protocols_token_tvl_v1' using 95 parque

In [3]:
# An aggregation that runs on top of the remote data.
# Takes ~5s to run, not to bad since we are only looking at dates starting 2024-10-01.
print(duckdb_ctx.client.sql(
f"""
SELECT 
    protocol_slug, 
    chain, 
    token, 
    sum(app_token_tvl_usd) AS tvl_usd, 
    count(distinct dt) AS num_dates
FROM {view1}
GROUP BY 1, 2, 3
ORDER BY 4 DESC
LIMIT 10
"""))

┌───────────────┬──────────┬─────────┬────────────────────┬───────────┐
│ protocol_slug │  chain   │  token  │      tvl_usd       │ num_dates │
│    varchar    │ varchar  │ varchar │       double       │   int64   │
├───────────────┼──────────┼─────────┼────────────────────┼───────────┤
│ binance-cex   │ Bitcoin  │ BTC     │  4668655572760.399 │        94 │
│ lido          │ Ethereum │ WETH    │  2839183272007.591 │        94 │
│ binance-cex   │ Binance  │ BNB     │ 1659644947899.1255 │        94 │
│ binance-cex   │ Ethereum │ USDT    │ 1543214467880.8442 │        94 │
│ bitfinex      │ Bitcoin  │ BTC     │ 1394980613157.7214 │        92 │
│ robinhood     │ Bitcoin  │ BTC     │ 1117340211544.2952 │        94 │
│ wbtc          │ Bitcoin  │ BTC     │ 1112880412688.9067 │        94 │
│ binance-cex   │ Ethereum │ WETH    │ 1045614936117.3087 │        94 │
│ okx           │ Bitcoin  │ BTC     │  937831051222.6948 │        94 │
│ eigenlayer    │ Ethereum │ WETH    │  875468307160.5481 │     

In [ ]:
## EXAMPLE 2: Dump the entire dataset locally

# A read with no parameters reads the entire dataset.
view2 = DefiLlama.PROTOCOLS_TOKEN_TVL.read()

# Dump to a local parquet file and create a new view that is 
# backed by the dumped file.
parquet_fileprefix = "defillama_protocols_token_tvl_v1_full"
view3 = dump_local_copy(view2, parquet_fileprefix=parquet_fileprefix)

print(view2)
print(view3)


2024-12-03 17:56:17 [info     ] registered view: 'defillama_protocols_token_tvl_v1' using uri wildcard 'gs://oplabs-tools-data-sink/defillama/protocols_token_tvl_v1/dt=*/out.parquet' filename=client.py lineno=53
┌──────────────────────────────────┐
│               name               │
│             varchar              │
├──────────────────────────────────┤
│ defillama_protocols_token_tvl_v1 │
└──────────────────────────────────┘

2024-12-03 17:56:18 [info     ] begin dump copy to '/Users/pedro/github/ethereum-optimism/op-analytics-alt/parquet/defillama_protocols_token_tvl_v1_full__20241203T225618.parquet' filename=localcopy.py lineno=20
2024-12-03 17:58:40 [info     ] done dump copy to '/Users/pedro/github/ethereum-optimism/op-analytics-alt/parquet/defillama_protocols_token_tvl_v1_full__20241203T225618.parquet' filename=localcopy.py lineno=22
┌────────────────────────────────────────────────────────┐
│                          name                          │
│                        v

In [5]:
## EXAMPLE 2 (continued)

## Use the local copy directly by providing the exact fetch timestamp
view4 = load_local_copy(parquet_fileprefix, timestamp="20241203T225618")
print(view4)

2024-12-03 17:59:43 [info     ] load copy from '/Users/pedro/github/ethereum-optimism/op-analytics-alt/parquet/defillama_protocols_token_tvl_v1_full__20241203T225618.parquet' filename=localcopy.py lineno=44
┌────────────────────────────────────────────────────────┐
│                          name                          │
│                        varchar                         │
├────────────────────────────────────────────────────────┤
│ defillama_protocols_token_tvl_v1                       │
│ defillama_protocols_token_tvl_v1_full__20241203T225618 │
└────────────────────────────────────────────────────────┘

defillama_protocols_token_tvl_v1_full__20241203T225618


In [ ]:
## Using view4, which is backed by the local copy results in fast queries.

print(duckdb_ctx.client.sql(
f"""
SELECT 
    min(dt)
FROM {view4}
"""))

┌────────────┐
│  min(dt)   │
│    date    │
├────────────┤
│ 2018-04-27 │
└────────────┘



In [8]:
# An aggregation that runs on top of the local copy.
# Takes ~3s to run (this includes ALL dates since 2018-04-27)
# Running this on the remote dataset would take longer.
print(duckdb_ctx.client.sql(
f"""
SELECT 
    protocol_slug, 
    chain, 
    token, 
    max(app_token_tvl_usd) AS max_tvl_usd, 
    arg_max(dt, app_token_tvl_usd) as max_tvl_usd_dt,
    count(distinct dt) AS num_dates
FROM {view4}
GROUP BY 1, 2, 3
ORDER BY 4 DESC
LIMIT 10
"""))

┌───────────────┬──────────────────┬─────────┬────────────────────┬────────────────┬───────────┐
│ protocol_slug │      chain       │  token  │    max_tvl_usd     │ max_tvl_usd_dt │ num_dates │
│    varchar    │     varchar      │ varchar │       double       │      date      │   int64   │
├───────────────┼──────────────────┼─────────┼────────────────────┼────────────────┼───────────┤
│ hotbit        │ Tron             │ TRX     │     728037762725.0 │ 2024-01-31     │       614 │
│ iron-finance  │ Polygon          │ TITAN   │ 244941123692.78592 │ 2021-06-16     │         4 │
│ binance-cex   │ Bitcoin          │ BTC     │  58700225047.75569 │ 2024-11-22     │       752 │
│ lido          │ Ethereum         │ WETH    │   40177918659.3112 │ 2024-03-12     │      1282 │
│ sushiswap     │ Polygon          │ TITAN   │  35896427732.67128 │ 2021-06-17     │       301 │
│ sushiswap     │ Harmony          │ RVRS    │  35486417432.64003 │ 2022-06-13     │       157 │
│ hex           │ staking     

In [9]:
## EXAMPLE 2 (continuud)

## Load last dumped local copy by omitting the timestamp parameter.
view5 = load_local_copy(parquet_fileprefix, timestamp=None)
print(view5)

2024-12-03 18:01:53 [info     ] load copy from '/Users/pedro/github/ethereum-optimism/op-analytics-alt/parquet/defillama_protocols_token_tvl_v1_full__20241203T225618.parquet' filename=localcopy.py lineno=44
┌────────────────────────────────────────────────────────┐
│                          name                          │
│                        varchar                         │
├────────────────────────────────────────────────────────┤
│ defillama_protocols_token_tvl_v1                       │
│ defillama_protocols_token_tvl_v1_full__20241203T225618 │
└────────────────────────────────────────────────────────┘

defillama_protocols_token_tvl_v1_full__20241203T225618


In [10]:

print(duckdb_ctx.client.sql(
f"""
SELECT 
    min(dt)
FROM {view5}
"""))

┌────────────┐
│  min(dt)   │
│    date    │
├────────────┤
│ 2018-04-27 │
└────────────┘



In [11]:
## EXAMPLE 3: Dump only a date range to a local parquet file


# Use a minimum date.
view6 = DefiLlama.PROTOCOLS_TOKEN_TVL.read(min_date="2024-06-01")

# Dump to a local parquet file and create a new view that is 
# backed by the dumped file.
parquet_fileprefix = "defillama_protocols_token_tvl_v1_post_0601"
view7 = dump_local_copy(view6, parquet_fileprefix=parquet_fileprefix)

2024-12-03 18:03:32 [info     ] querying markers for 'defillama/protocols_token_tvl_v1' DateFilter(min_date=datetime.date(2024, 6, 1), max_date=None, datevals=None) filename=dailydata.py lineno=95
2024-12-03 18:03:32 [info     ] 201 markers found              filename=dailydata.py lineno=109
2024-12-03 18:03:32 [info     ] 186 distinct paths             filename=dailydata.py lineno=115
2024-12-03 18:03:33 [info     ] registered view: 'defillama_protocols_token_tvl_v1' using 186 parquet paths filename=client.py lineno=53
┌────────────────────────────────────────────────────────┐
│                          name                          │
│                        varchar                         │
├────────────────────────────────────────────────────────┤
│ defillama_protocols_token_tvl_v1                       │
│ defillama_protocols_token_tvl_v1_full__20241203T225618 │
└────────────────────────────────────────────────────────┘

2024-12-03 18:03:33 [info     ] begin dump copy to '/Users/p

In [12]:
# Use view7 which is backed by a local copy of the date-filtered data.

print(duckdb_ctx.client.sql(
f"""
SELECT 
    protocol_slug, 
    chain, 
    token, 
    max(app_token_tvl_usd) AS max_tvl_usd, 
    arg_max(dt, app_token_tvl_usd) as max_tvl_usd_dt,
    count(distinct dt) AS num_dates
FROM {view7}
GROUP BY 1, 2, 3
ORDER BY 4 DESC
LIMIT 10
"""))

┌───────────────┬──────────┬─────────┬───────────────────┬────────────────┬───────────┐
│ protocol_slug │  chain   │  token  │    max_tvl_usd    │ max_tvl_usd_dt │ num_dates │
│    varchar    │ varchar  │ varchar │      double       │      date      │   int64   │
├───────────────┼──────────┼─────────┼───────────────────┼────────────────┼───────────┤
│ binance-cex   │ Bitcoin  │ BTC     │ 58700225047.75569 │ 2024-11-22     │       186 │
│ lido          │ Ethereum │ WETH    │ 36626823339.87432 │ 2024-12-02     │       186 │
│ binance-cex   │ Ethereum │ USDT    │ 21849070391.40374 │ 2024-12-03     │       186 │
│ binance-cex   │ Binance  │ BNB     │ 20804707506.85196 │ 2024-06-06     │       186 │
│ bitfinex      │ Bitcoin  │ BTC     │ 17063769859.88684 │ 2024-12-02     │       186 │
│ wbtc          │ Bitcoin  │ BTC     │ 14445877224.43124 │ 2024-11-23     │       109 │
│ robinhood     │ Bitcoin  │ BTC     │ 14082757853.59867 │ 2024-12-02     │       186 │
│ binance-cex   │ Tron     │ USD

In [13]:
## EXAMPLE 4: Read metadata. 
# 
# The metadata df is written out entirely for each date
# So th read it on must pick a single date.

metadata_view1 = DefiLlama.PROTOCOLS_METADATA.read(date_range_spec="@20241126:+1")
duckdb_ctx.client.sql(f"SELECT * FROM {metadata_view1} LIMIT 10")

2024-12-03 18:04:52 [info     ] querying markers for 'defillama/protocols_metadata_v1' DateFilter(min_date=None, max_date=None, datevals=[datetime.date(2024, 11, 26)]) filename=dailydata.py lineno=95
2024-12-03 18:04:52 [info     ] 1 markers found                filename=dailydata.py lineno=109
2024-12-03 18:04:52 [info     ] 1 distinct paths               filename=dailydata.py lineno=115
2024-12-03 18:04:53 [info     ] registered view: 'defillama_protocols_metadata_v1' using 1 parquet paths filename=client.py lineno=53
┌─────────────────────────────────────────────────────────────┐
│                            name                             │
│                           varchar                           │
├─────────────────────────────────────────────────────────────┤
│ defillama_protocols_metadata_v1                             │
│ defillama_protocols_token_tvl_v1                            │
│ defillama_protocols_token_tvl_v1_full__20241203T225618      │
│ defillama_protocols_toke

┌─────────────────┬─────────────────┬───────────────────┬─────────────────┬─────────────────┬───────────────────────┬────────────┐
│  protocol_name  │  protocol_slug  │ protocol_category │ parent_protocol │ wrong_liquidity │ misrepresented_tokens │     dt     │
│     varchar     │     varchar     │      varchar      │     varchar     │     boolean     │        boolean        │    date    │
├─────────────────┼─────────────────┼───────────────────┼─────────────────┼─────────────────┼───────────────────────┼────────────┤
│ Binance CEX     │ binance-cex     │ CEX               │ binance-cex     │ NULL            │ NULL                  │ 2024-11-26 │
│ Lido            │ lido            │ Liquid Staking    │ lido            │ NULL            │ NULL                  │ 2024-11-26 │
│ Bitfinex        │ bitfinex        │ CEX               │ bitfinex        │ NULL            │ NULL                  │ 2024-11-26 │
│ OKX             │ okx             │ CEX               │ okx             │ NULL   